<a href="https://colab.research.google.com/github/CharlemagneBrain/Tf_Transform_TFX/blob/main/C2_W2_Lab_1_Simple_Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple ingénierie des fonctionnalités

Dans ce labo, vous pratiquerez concrètement l'utilisation de la bibliothèque Tensorflow Transform (ou tf.Transform). Cela permet de montrer ce qui se passe en coulisses lorsque vous utilisez le composant TFX Transform dans un pipeline TFX dans les prochains labs. Les extraits de code et les discussions principales proviennent de ce notebook officiel de TensorFlow, mais nous avons développé quelques points clés.

La préparation des données est souvent nécessaire dans les projets de ML car les données brutes ne sont pas encore dans un format approprié pour l'entraînement d'un modèle. Ne pas le faire aboutit généralement à ce que le modèle ne converge pas ou ait de mauvaises performances. Certaines transformations standard comprennent la normalisation des valeurs de pixels, la mise en buckets, l'encodage one-hot, et ainsi de suite. Par conséquent, ces mêmes transformations doivent également être effectuées lors de l'inférence pour s'assurer que le modèle calcule les prédictions correctes.

Avec Tensorflow Transform, vous pouvez prétraiter les données en utilisant le même code à la fois pour l'entraînement d'un modèle et pour les inférences en production. Il fournit plusieurs fonctions utilitaires pour les tâches de prétraitement courantes, notamment la création de fonctionnalités qui nécessitent un passage complet sur l'ensemble de données d'entraînement. Les sorties sont les fonctionnalités transformées et un graphe TensorFlow que vous pouvez utiliser à la fois pour l'entraînement et pour le service. L'utilisation du même graphe pour l'entraînement et le service peut empêcher l'incohérence des fonctionnalités, car les mêmes transformations sont appliquées dans les deux phases.

Pour cet exercice d'introduction, vous allez parcourir le "Hello World" de l'utilisation de TensorFlow Transform pour prétraiter les données d'entrée. Comme vous l'avez vu en classe, les étapes principales sont les suivantes :

1. Collectez les données brutes
2. Définissez les métadonnées
3. Créez une fonction de prétraitement
4. Générez un graphe constant avec les transformations requises
Commençons !

## Importation des librairies

In [ ]:
import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_transform.beam as tft_beam

from tensorflow_transform.tf_metadata import dataset_metadata
from tensorflow_transform.tf_metadata import schema_utils

import pprint
import tempfile

print(f'TensorFlow version: {tf.__version__}')
print(f'TFX Transform version: {tft.__version__}')

## Collecte des données brutes

Tout d'abord, vous devez charger vos données. Pour des raisons de simplicité, nous n'utiliserons pas d'ensemble de données réel dans cet exercice. Vous le ferez dans le prochain laboratoire. Pour l'instant, vous utiliserez simplement des données fictives afin de pouvoir inspecter les transformations plus facilement.

In [ ]:
# define sample data
raw_data = [
      {'x': 1, 'y': 1, 's': 'hello'},
      {'x': 2, 'y': 2, 's': 'world'},
      {'x': 3, 'y': 3, 's': 'hello'}
  ]

## Définir les métadonnées

Ensuite, vous allez définir les métadonnées. Elles contiennent le schéma qui indique les types de chaque colonne de caractéristiques (ou clé) dans `raw_data`. Vous devez prendre note de certaines choses :

* La fonction de transformation attend que les métadonnées soient emballées dans un objet [DatasetMetadata](https://github.com/tensorflow/transform/blob/master/tensorflow_transform/tf_metadata/dataset_metadata.py#L23). 
* Le constructeur de la classe `DatasetMetadata` attend un type de données [Schema protocol buffer](https://github.com/tensorflow/metadata/blob/master/tensorflow_metadata/proto/v0/schema.proto#L46). Vous pouvez utiliser la méthode [schema_from_feature_spec()](https://github.com/tensorflow/transform/blob/master/tensorflow_transform/tf_metadata/schema_utils.py#L36) pour le générer à partir d'un dictionnaire.
* Pour construire ledit dictionnaire, vous utiliserez les clés/noms de colonnes de `raw_data` et assignerez un [FeatureSpecType](https://github.com/tensorflow/transform/blob/master/tensorflow_transform/common_types.py#L29) comme valeurs. Cela vous permet de spécifier si l'entrée est de longueur fixe ou variable (en utilisant les classes [tf.io](https://www.tensorflow.org/api_docs/python/tf/io)), ainsi que de définir la forme et le type de données.

Voyez comment cela est mis en œuvre dans la cellule ci-dessous.


In [ ]:
# définir le schéma comme un objet DatasetMetadata
raw_data_metadata = dataset_metadata.DatasetMetadata(
    
    # utiliser la fonction de commodité pour construire un Schema protobuf
    schema_utils.schema_from_feature_spec({
        
        # 
        'y': tf.io.FixedLenFeature([], tf.float32),
        'x': tf.io.FixedLenFeature([], tf.float32),
        's': tf.io.FixedLenFeature([], tf.string),
    }))

In [ ]:
# preview the schema
print(raw_data_metadata._schema)

## Créer une fonction de prétraitement

La _fonction de prétraitement_ est le concept le plus important de `tf.Transform`. Une fonction de prétraitement est l'endroit où la transformation de l'ensemble de données se produit réellement. Il accepte et renvoie un dictionnaire de tenseurs, où un tenseur signifie un <a target='_blank' href='https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/Tensor'>< code>Tensor</code></a> ou <a target='_blank' href='https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/SparseTensor'><code> SparseTensor</code></a>. Il existe deux principaux groupes d'appels d'API qui forment généralement le cœur d'une fonction de prétraitement :

1. **TensorFlow Ops :** Toute fonction qui accepte et renvoie des tenseurs. Ceux-ci ajoutent des opérations TensorFlow au graphique qui transforme les données brutes en données transformées, un vecteur de caractéristiques à la fois. Ceux-ci seront exécutés pour chaque exemple, pendant l'entraînement et le service.
2. **Analyseurs de transformation TensorFlow :** N'importe lequel des analyseurs fournis par `tf.Transform`. Les analyseurs acceptent et renvoient également les tenseurs, mais contrairement aux opérations TensorFlow, ils ne s'exécutent qu'une seule fois pendant la formation et effectuent généralement une passe complète sur l'ensemble des données de formation. Ils créent des <a target='_blank' href='https://www.tensorflow.org/versions/r1.15/api_docs/python/tf/constant'>constantes tensorielles</a>, qui sont ajoutées à votre graphique . Par exemple, `tft.min` calcule le minimum d'un tenseur sur l'ensemble de données d'apprentissage.

*Attention : Lorsque vous appliquez votre fonction de prétraitement à la diffusion d'inférences, les constantes créées par les analyseurs lors de l'entraînement ne changent pas. Si vos données comportent des composantes de tendance ou de saisonnalité, planifiez en conséquence.*

Vous pouvez voir les fonctions disponibles pour transformer vos données [ici](https://www.tensorflow.org/tfx/transform/api_docs/python/tft).

In [ ]:
def preprocessing_fn(inputs):
    """Preprocess input columns into transformed columns."""
    
    # extract the columns and assign to local variables
    x = inputs['x']
    y = inputs['y']
    s = inputs['s']
    
    # data transformations using tft functions
    x_centered = x - tft.mean(x)
    y_normalized = tft.scale_to_0_1(y)
    s_integerized = tft.compute_and_apply_vocabulary(s)
    x_centered_times_y_normalized = (x_centered * y_normalized)
    
    # return the transformed data
    return {
        'x_centered': x_centered,
        'y_normalized': y_normalized,
        's_integerized': s_integerized,
        'x_centered_times_y_normalized': x_centered_times_y_normalized,
    }

## Générer un graphique constant avec les transformations requises

Vous êtes maintenant prêt à tout assembler et à transformer vos données. Comme TFDV la semaine dernière, Tensorflow Transform utilise également [Apache Beam](https://beam.apache.org/) pour l'extensibilité et la flexibilité du déploiement. Comme vous le verrez ci-dessous, Beam utilise l'opérateur pipe (`|`) pour empiler les différentes étapes du pipeline. Dans ce cas, il suffit d'envoyer les données (et les métadonnées) à la classe [AnalyzeAndTransformDataset](https://www.tensorflow.org/tfx/transform/api_docs/python/tft_beam/AnalyzeAndTransformDataset) et d'utiliser la fonction de prétraitement définie ci-dessus pour transformer les données.

Pour un examen plus approfondi de la syntaxe de Beam pour les pipelines de transformation, vous pouvez vous référer à la documentation [ici](https://beam.apache.org/documentation/programming-guide/#applying-transforms) et essayer le petit Colab [ici](https://colab.research.google.com/github/apache/beam/blob/master/examples/notebooks/get-started/try-apache-beam-py.ipynb#scrollTo=J5HMFSzD8O2U).

*Note : Vous pouvez ignorer sans risque l'avertissement concernant les args non séparables affiché après l'exécution de la cellule ci-dessous.*

In [ ]:
# Ignore the warnings
tf.get_logger().setLevel('ERROR')

# a temporary directory is needed when analyzing the data
with tft_beam.Context(temp_dir=tempfile.mkdtemp()):
    
    # define the pipeline using Apache Beam syntax
    transformed_dataset, transform_fn = (
        
        # analyze and transform the dataset using the preprocessing function
        (raw_data, raw_data_metadata) | tft_beam.AnalyzeAndTransformDataset(
            preprocessing_fn)
    )

# unpack the transformed dataset
transformed_data, transformed_metadata = transformed_dataset

# print the results
print('\nRaw data:\n{}\n'.format(pprint.pformat(raw_data)))
print('Transformed data:\n{}'.format(pprint.pformat(transformed_data)))

## Is this the right answer?
Previously, you used `tf.Transform` to do this:
```
x_centered = x - tft.mean(x)
y_normalized = tft.scale_to_0_1(y)
s_integerized = tft.compute_and_apply_vocabulary(s)
x_centered_times_y_normalized = (x_centered * y_normalized)
```
#### x_centered
With input of `[1, 2, 3]` the mean of `x` is 2, and you subtract it from `x` to center your `x` values at 0.  So the result of `[-1.0, 0.0, 1.0]` is correct.
#### y_normalized
Next, you scaled your `y` values between 0 and 1.  Your input was `[1, 2, 3]` so the result of `[0.0, 0.5, 1.0]` is correct.
#### s_integerized
You mapped your strings to indexes in a vocabulary, and there were only 2 words in your vocabulary ("hello" and "world").  So with input of `["hello", "world", "hello"]` the result of `[0, 1, 0]` is correct.
#### x_centered_times_y_normalized
You created a new feature by crossing `x_centered` and `y_normalized` using multiplication.  Note that this multiplies the results, not the original values, and the new result of `[-0.0, 0.0, 1.0]` is correct.

### Wrap Up

In this lab, you went through the fundamentals of using Tensorflow Transform to turn raw data into features. This code can be used to transform both the training and serving data. However, the code can be quite complex if you'll be using this as a standalone library to build a pipeline (see this [notebook](https://www.tensorflow.org/tfx/tutorials/transform/census) for reference). Now that you know what is going on under the hood, you can use a higher-level set of tools like [Tensorflow Extended](https://www.tensorflow.org/tfx) to simplify the process. This will abstract some of the steps you did here like manually defining schemas and using `tft_beam` functions. It will also leverage other libraries, such as TFDV, to perform other processes in the usual machine learning pipeline like detecting anomalies. You will get to see these in the next lab.